In [3]:
import pandas as pd
import numpy as np
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import copy


import openpyxl
cf.go_offline(connected= True)

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [20]:
PIB_dict = {'A' : 160, 
'B' : 200,
'C' : 240,
'D' : 220 ,
'E' : 300,
'F' : 360,
'G' : 280,
'H' : 400,
'I' : 350,
'J' : 300
}

In [21]:
import openpyxl
from pandas import ExcelWriter


class Mereged:
    
    def __init__(self,file_input, file_output):
        self.file_input = file_input
        self.file_output = file_output
    
    
    
    def make_merge(self):
        lst_xlsx = []

        wb=openpyxl.load_workbook(str(self.file_input)+'.xlsx')
        sheet_name = wb.get_sheet_names()

        for ww in sheet_name:

            sheet = wb.get_sheet_by_name(ww)
            print(sheet)
            ws = sheet

            f = pd.DataFrame(ws.values)
            f2 = f.copy()
            f2.loc[0,0] = 'first'
            f2.columns = f2.loc[0,:]
            f2 = f2.set_index('first').iloc[1:,:]
            f2.index.name = None

            if 'Gross_requirements' in f2.index:
                f3 = f2.copy()
                PIB_first = PIB_dict[ww]
                f3.loc['Projected_Inventory_Balanance',0] = PIB_first

                f4 = f3.copy()
                lst_col = f4.columns

                lst_PIB = []


                num = 1
                for idx,i in enumerate(zip(f4.loc['Gross_requirements',:], f4.loc['Projected_Inventory_Balanance'])):
                    if idx == 0:
                        pass
                    else:
                        delta =  f4.loc['Projected_Inventory_Balanance',lst_col[idx-1]] - i[0]

                        f4.loc['Projected_Inventory_Balanance',lst_col[idx]] = delta


                lst_Net = []
                num = 0

                for idx,i in enumerate(zip(f4.loc['Gross_requirements',:], f4.loc['Projected_Inventory_Balanance'])):

                    if i[1] >= 0:
                        lst_Net.append(0)

                    else:
                        if num == 0:
                            lst_Net.append(abs(f4.loc['Projected_Inventory_Balanance',lst_col[idx-1]] - i[0]))
                        else:
                            lst_Net.append(f4.loc['Gross_requirements',lst_col[idx]])
                        num += 1


                f4.loc['Net_requirements',:] = lst_Net



                lst_PIB = []
                for i in f4.loc['Projected_Inventory_Balanance',:]:
                    if i < 0:
                        lst_PIB.append(0)
                    else:
                        lst_PIB.append(i)

                f4.loc['Projected_Inventory_Balanance',:] = lst_PIB


                A, B = f4.loc['Planned_receipts',:] , f4.loc['Planned_order_release',:]


                for idx, i in enumerate(A):
                    if i > 0:
                        alpha = idx 
                    else:
                        pass


                for idx, i in enumerate(B):
                    if i > 0:
                        beta = idx 
                    else:
                        pass

                difference = abs(alpha - beta)



                if sum(f4.loc['Net_requirements', : difference+1]) > 0:
                    f4.loc['scheduled_receipts',:difference+1] = f4.loc['Net_requirements', : difference+1]
                    f4.loc['Net_requirements', : difference+1] = 0


                f5 = f4.copy()

                f5.loc['Planned_receipts',:] =  f5.loc['Net_requirements',:].copy()
                f5.loc['Planned_order_release',:] = f5.loc['Planned_receipts',difference:].tolist() + [0]*difference



            else:
                f5 = f2.copy()

            lst_xlsx.append((ww,f5))
            
        return lst_xlsx
    
    
    def make_file(self,lst_xlsx):
    
        writer = pd.ExcelWriter(str(self.file_output) +'.xlsx',engine= 'openpyxl')
        
        for i,j in lst_xlsx:
            j.to_excel(writer, sheet_name = str(i))
            
            writer.save()
            
        writer.close()
        return lst_xlsx
    
    
    def Final(self):
        
        a = self.make_file(self.make_merge())

        return a

In [22]:
A = Mereged('ALL','output')

A.Final()



D:\anaconda\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning:

Call to deprecated function get_sheet_names (Use wb.sheetnames).

D:\anaconda\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning:

Call to deprecated function get_sheet_by_name (Use wb[sheetname]).



<Worksheet "D">
<Worksheet "A">
<Worksheet "E">
<Worksheet "J">
<Worksheet "G">
<Worksheet "B">
<Worksheet "I">
<Worksheet "C">
<Worksheet "F">
<Worksheet "H">


[('D',
  0                                0    1    2    3    4    5    6    7    8  \
  Gross_requirements               0  160  120  150  150  150  200  200  200   
  scheduled_receipts               0    0   60    0    0    0    0    0    0   
  Projected_Inventory_Balanance  220   60    0    0    0    0    0    0    0   
  Net_requirements                 0    0    0  150  150  150  200  200  200   
  Planned_receipts                 0    0    0  150  150  150  200  200  200   
  Planned_order_release            0    0  150  150  150  200  200  200  200   
  
  0                                9   10  11  12  
  Gross_requirements             200  200   0   0  
  scheduled_receipts               0    0   0   0  
  Projected_Inventory_Balanance    0    0   0   0  
  Net_requirements               200  200   0   0  
  Planned_receipts               200  200   0   0  
  Planned_order_release          200    0   0   0  ),
 ('A',
  0               0    1               2    3    4    5  

In [26]:
writer.close()